### OBTENCION DE DATOS API MERCADOLIBRE

#### Importacion de librerias necesarias

In [1]:
import json
import requests
from urllib.parse import quote
import pandas as pd
from scripts.Publication import CarPublicationBuilder

#### Obtencion de publicaciones

El objetivo de esta seccion es obtener un listado de publicaciones de "vehiculos" desde la API de **MercadoLibre (https://developers.mercadolibre.com.ar/)** utilizando la libreria **requests** de python.<br>En primer lugar se crea una lista(**cars**) que aloja diferentes modelos de vehiculos disponibles en el mercado automotriz, posteriormente mediante el uso de la libreria **urllib** y su metodo **quote** se codifica la lista **cars** a fin de transformar ciertos caracteres para mantener la integridad de la URL al momento de realizar la request.

In [2]:
#LISTA DE VEHICULOS
cars = [    'Volkswagen gol','Volkswagen gol trend', 'Volkswagen up', 'Volkswagen bora', 'Volkswagen vento', 'Volkswagen polo', 'Volkswagen amarok', 'Volkswagen golf',
            'Chevrolet corsa', 'Chevrolet onix', 'Chevrolet agile', 'Chevrolet prisma', 'Chevrolet cruze', 'Chevrolet aveo', 'Chevrolet tracker',
            'Renault clio', 'Renault sandero', 'Renault fluence', 'Renault logan', 'Reanault partner', 'Renault capture',
            'Peugeot 206', 'Peugeot 207', 'Peugeot 208', 'Peugeot 307','Peugeot 308', 'Peugeot 407', 'Peugeot 408',
            'Fiat cronos', 'Fiat argos', 'Fiat toro', 'Fiat palio', 'Fiat siena', 'Fiat uno',                   
            'Ford fiesta', 'Ford Ecosport', 'Ford ka', 'Ford ranger', 'Ford focus',
            'Toyota etios', 'Toyota hilux', 'Toyota corolla', 'Toyota yaris', 
            'Honda civic', 'Honda city', 'Honda cr-v']

#SE CODIFICAN LOS STRINGS DE LA LISTA PARA USARSE EN REQUESTS
cars_codif = []
for car in cars:
    cars_codif.append(quote(car))

Con la finalidad de realizar una request por cada modelo de automovil en la lista, se crea la funcion **get_publication()**. Esta funcion  devuelve una lista de diccionarios, cada uno de estos diccionarios representa una publicacion de **MercadoLibre**. Dentro de la logica de la funcion se establece un maximo de 350 publicaciones por vehiculo.

In [3]:
#FUNCION PARA REALIZAR RESQUESTS Y OBTENER PUBLICACIONES SEGUN EL MODELO DEL VEHICULO
def get_publications(car : str) -> list:
    publication = []
    url = f'https://api.mercadolibre.com/sites/MLA/search?q={car}'
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        results = data.get('paging', dict).get('total', None)  #TOTAL RESULTADOS EXISTENTES
        offset = 0 #POSICION DE INICIO PARA LA REQUEST

        if 'results' in data:
            for item in data['results']:
                publication.append(item)

            while (results > offset) and (results > 50):
                offset +=50 
                if offset == 350: break 
                url = f'https://api.mercadolibre.com/sites/MLA/search?q={car}&offset={offset}'
                response = requests.get(url)
                data = response.json()
                if 'results' in data:
                    for item in data['results']:
                        publication.append(item)  

        return publication                    
    else:
        return publication

Se crea una la lista **publications_list** la cual alojará los diccionarios obtenidos de la funcion **get_publication()**. Acto seguido ejecuta un ciclo for a fines de recorrer la lista **cars_codif**, ejecutar la funcion **get_publication()** por cada uno de los elementos en la lista, luego los resultados obtenidos son alojados en **publications_list**

In [4]:
#LISTA PARA GUARDAR LAS PUBLICACIONES OBTENIDAS EN EL REQUEST
publications_list = []

#CICLO PARA REALIZAR UN REQUEST POR CADA MODELO DE AUTO EN CARS_CODIF
for car in cars_codif:
    publications = get_publications(car)
    for publication in publications:
        publications_list.append(publication)

print(f'La cantidad de publicaciones obtenidas es: {len(publications_list)}')

La cantidad de publicaciones obtenidas es: 14799


#### Extraccion de los datos de interes

Con la finalidad de estructurar los datos extraidos de la API anteriormente se crea la clase **CarPublicaction()** la cual aloja en sus atributos los datos de interes de las publicaciones que se obtubieron. Tambien se crea la clase **CarPublicationBuilder()** cuya responsabilidad es extraer de un diccionario(publicacion) los datos de interes y devolver una instancia de la clase **CarPublicaction()** con sus atributos seteados

Mediante un ciclo for se recorre **publications_list** con la finalidad de ejecutar una instancia de la clase **CarPublicationBuilder()** y obtener una objeto de tipo **CarPublicaction()** por cada diccionario en **publications_list** 

In [5]:
#LISTA PARA GUARDAR LOS DATOS CON EL FORMATO DE INTERES 
car_publications = []

#CICLO PARA EXTRAER LOS DATOS DE INTERES DE LAS PUBLICACIONES
for item in publications_list:
    publication = CarPublicationBuilder(item).build()
    car_publications.append(publication)

print(f'Cantidad de registros obtenidos: {len(car_publications)}')

Cantidad de registros obtenidos: 14799


Se guarda el resultado obtenido en un archivo **JSON** de manera local

In [6]:
#SE GUARDAN LOS DATOS DE INTERES EN UN ARCHIVO JSON
with open('MeliCarPublicactions.json','w' ) as file:
    json.dump(car_publications, file, indent=4, separators=(',',':'))

Utilizando **Pandas** se puede leer la lista de diccionarios (**car_publications**) para obtener un **Dataframe** y manipular los datos segun la necesidad.

In [7]:
df = pd.DataFrame(car_publications)
df.head(3)

,id,category_id,title,condition,car_year,brand,model,version,engine,engine_power,...,transmission,currency,price,seller_id,seller_nickname,is_car_shop,seller_country,seller_state,seller_city,seller_neighborhood
0,MLA1767503710,MLA1744,Volkswagen Gol Power,Usado,2008,Volkswagen,Gol,1.9 Sd Power 701,1.9,64 hp,...,Manual,USD,5200,1601211580,TIERRADEAUTOS,False,Argentina,Bs.As. G.B.A. Sur,Avellaneda,Villa Domínico
1,MLA1776550792,MLA1744,Volkswagen Gol 2012 1.4 Power Ps+ac 83cv,Usado,2012,Volkswagen,Gol,1.4 Power Ps+ac 83cv,1.4,83 hp,...,Manual,USD,6250,146472306,ARIEL_RICCI,False,Argentina,Capital Federal,Capital Federal,None
2,MLA1776084372,MLA1744,Volkswagen Gol 1.6 I Power /// 2010 - 140.000km,Usado,2010,Volkswagen,Gol,1.6 I Power 701,1.6,92 hp,...,Manual,USD,6000,77745224,CAR.ANT,False,Argentina,Bs.As. G.B.A. Norte,San Isidro,None
